In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install flask
!pip install flask-cors
!pip install pyngrok
!ngrok config add-authtoken 2oxssltc9xj7I8qwbsmxAoxk2DG_7ghj78vBXdk5nvNKW7rqH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from pyngrok import ngrok
url = ngrok.connect(5000)
print("Ngrok URL: ", url)

Ngrok URL:  NgrokTunnel: "https://6245-34-125-48-39.ngrok-free.app" -> "http://localhost:5000"


In [7]:
!pip install flask flask-cors transformers pillow torch

In [8]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import torch
import io

# Create a Flask application
app = Flask(__name__)
CORS(app)

# Load the model and processor
model_name = "Thao2202/vit-Facial-Expression-Recognition"
model = AutoModelForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

# Ensure labels_list matches the model's labels
labels_list = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

@app.route("/", methods=["POST"])
def predict():
    try:
        # Check if no file is in the request
        if 'file' not in request.files:
            return jsonify({"error": "No file found"}), 400

        file = request.files['file']

        # Process the image
        try:
            img = Image.open(io.BytesIO(file.read())).convert("RGB")
        except Exception as e:
            return jsonify({"error": f"Invalid image file: {str(e)}"}), 400

        # Preprocess the image
        inputs = processor(images=img, return_tensors="pt")

        # Make predictions
        try:
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                predicted_class = logits.argmax(-1).item()
                confidence = torch.softmax(logits, dim=-1)[0][predicted_class].item()
        except Exception as e:
            return jsonify({"error": f"Model prediction error: {str(e)}"}), 500

        # Validate predicted_class and map to label
        if predicted_class < len(labels_list):
            label = labels_list[predicted_class]
        else:
            return jsonify({"error": "Invalid predicted class index"}), 500

        # Return the result
        return jsonify({
            "Label": label,
            "confidence": round(confidence, 4)
        })

    except Exception as e:
        return jsonify({"error": f"Internal server error: {str(e)}"}), 500

@app.route("/")
def home():
    return "Emotion Detection Model API is running!"

# Run the Flask application
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:56:12] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 15:02:20] "POST / HTTP/1.1" 200 -
